In [1]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from keras.layers import Merge
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

from yolo import yolo_filter_boxes, iou, yolo_non_max_suppression, yolo_eval, predict, class_names, anchors

C:\ProgramData\Anaconda3\envs\qda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
keras.__version__

'2.1.5'

In [3]:
sess = K.get_session()

In [4]:
yolo_model = load_model("model_data/yolo.h5")

C:\ProgramData\Anaconda3\envs\qda\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
yolo_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 608, 608, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 608, 608, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [6]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

In [7]:
def pipeline(image):
    #img = Image.fromarray(image)
    scores, boxes, classes = yolo_eval(yolo_outputs)
    image_pred =  predict(sess, image, scores, boxes, classes,yolo_model)
    #open_cv_image = np.array(pil_img)
    #open_cv_image = open_cv_image[:, :, ::-1].copy()
    return image_pred
    

In [8]:

#cv2.imshow('img',img)
#cv2.waitKey(500)

In [10]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'test_video_output.mp4'
clip1 = VideoFileClip("test_video.mp4")
white_clip = clip1.fl_image(pipeline)
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="1280" height="720" controls>
  <source src="{0}">
</video>
""".format(white_output))


[MoviePy] >>>> Building video test_video_output.mp4
[MoviePy] Writing video test_video_output.mp4


 97%|███████████████████████████████████████████████████████████████████████████████████████████████████▍  | 38/39 [01:15<00:02,  2.10s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_video_output.mp4 

Wall time: 1min 16s
